In [ ]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from ipywidgets import Select, HBox, VBox, Layout, Button, HTML
import bqplot.pyplot as plt
from bqplot import *
from bqplot.interacts import *
from bqplot.colorschemes import *

In [ ]:
yc_df = pd.read_csv('../data/yc_time_series.csv', index_col=0, parse_dates=True)
yc_data = yc_df.dropna(axis=0).values
dates = yc_df.index.values
tenors = list(yc_df.columns)

In [ ]:
k = 1
pca_model = PCA(n_components=k)

In [ ]:
ae = Sequential()
ae.add(Dense(8, input_shape=(2, ), activation='tanh'))
ae.add(Dense(5, activation='tanh'))
ae.add(Dense(3, activation='tanh'))
ae.add(Dense(k, activation='linear'))
ae.add(Dense(3, activation='relu'))
ae.add(Dense(5, activation='relu'))
ae.add(Dense(8, activation='relu'))
ae.add(Dense(2, activation='linear'))
optim = Adam()
ae.compile(loss='mse', optimizer=optim)

In [ ]:
# time series chart of yields with intsel
xs1, ys1 = DateScale(), LinearScale(min=0., max=0.06)
time_series = Lines(x=dates, y=yc_data.T, scales={'x': xs1, 'y': ys1}, colors=CATEGORY20c, 
                    labels=tenors, labels_visibility='label', apply_clip=False)
xax1 = Axis(scale=xs1, label='Dates', grid_lines='solid')
yax1 = Axis(scale=ys1, orientation='vertical', tick_format='.1%', grid_lines='solid', 
            label_location='end', label_offset='-2ex')
intsel = BrushIntervalSelector(scale=xs1, marks=[time_series])
ts_fig = Figure(marks=[time_series], axes=[xax1, yax1], interaction=intsel, 
                layout=Layout(width='1400px', min_height='450px'))


# scatter of rates of two selected tenors from dropdown
xs2, ys2 = LinearScale(), LinearScale()
cs = DateColorScale()

common_scat_args = dict(x=[], y=[], default_size=32, stroke='black')
scat = Scatter(**common_scat_args, color=[], scales={'x': xs2, 'y': ys2, 'color': cs})
               
xax2 = Axis(scale=xs2, label='2Y', tick_format='.1%', grid_lines='solid')
yax2 = Axis(scale=ys2, orientation='vertical', tick_format='.1%', grid_lines='solid', 
            label='10Y')
cax = ColorAxis(scale=cs, label='Dates', tick_format='%m/%Y', 
                num_ticks=5, orientation='vertical', side='right')

pca_scat = Scatter(**common_scat_args, default_colors=['deepskyblue'], 
                   scales={'x': xs2, 'y': ys2},
                   labels=['PCA'], display_legend=True)

ae_scat = Scatter(**common_scat_args, default_colors=['magenta'],
                  scales={'x': xs2, 'y': ys2},
                  labels=['AE'], display_legend=True)

fig = Figure(marks=[scat, pca_scat, ae_scat], axes=[xax2, yax2, cax],
             animation_duration=1000, title='2Y vs 10Y Yields',
             legend_location='top-left',
             fig_margin=dict(top=60, right=120, bottom=40, left=60),
             layout=Layout(width='900px', height='550px'))

fit_pca_ae_btn = Button(description='Fit PCA/AE', button_style='success')
fit_pca_ae_btn.on_click(lambda btn: fit_pca_ae(None))

busy_icon = HTML('<span class="fa-layers fa-fw fa-1x">' + 
                 '<i class="fa fa-spinner fa-3x fa-spin" style="position:absolute; top:-5px; left: 5px"> </i></span>',
                 layout=Layout(visibility='hidden'))

def update_scatter(*args):
    global X
    selected = time_series.selected
    if not intsel.brushing:
        if not selected:
            s, e = 0, yc_data.shape[0] - 1
        else:
            s, e = selected[0], selected[-1]
        

        # clear PCA and AE scatters
        with pca_scat.hold_sync():
            pca_scat.x = []
            pca_scat.y = []
        
        with ae_scat.hold_sync():
            ae_scat.x = []
            ae_scat.y = []
        
        # X matrix to input into models
        X = yc_df[s:e][['2Y', '10Y']].values
        
        # make sure number of scatter points dont exceed
        # 1000 to go easy on scatter rendering
        n = len(yc_df[s:e])
        cap = 1000
        step = np.int(np.ceil(n / cap))
        with scat.hold_sync():
            scat.x = X[::step, 0]
            scat.y = X[::step, 1]
            scat.color = yc_df[s:e:step].index.values

def fit_pca_ae(*args):
    global X
    try:
        busy_icon.layout.visibility = 'visible'
        
        # fit pca on X matrix
        pca_coeffs = pca_model.fit_transform(X)
        pca_preds = pca_model.inverse_transform(pca_coeffs)
            
        # fit AE on X matrix
        _ = ae.fit(X, X, batch_size=64, epochs=100, verbose=0)
        ae_preds = ae.predict(X)    

        n = len(X)
        cap = 1000
        step = np.int(np.ceil(n / cap))
        with pca_scat.hold_sync():
            pca_scat.x = pca_preds[::step, 0]
            pca_scat.y = pca_preds[::step, 1]
        
        with ae_scat.hold_sync():
            ae_scat.x = ae_preds[::step, 0]
            ae_scat.y = ae_preds[::step, 1]
    finally:
        busy_icon.layout.visibility = 'hidden'

intsel.observe(update_scatter, 'brushing')

VBox(
    [ts_fig, 
     HBox([fig, fit_pca_ae_btn, busy_icon], layout=Layout(margin='40px 0px 0p 0px'))]
)